# Usage

First, importing:

In [ ]:
from solarmach import SolarMACH, print_body_list, get_sw_speed, get_gong_map, calculate_pfss_solution

---
## 1. Minimal example <a class="anchor" id="Minimal_example"></a>

Necessary options are a list of wanted spacecraft/bodies and the date of interest.

Note that since version 0.2.3, you don't need to provide a list with solar wind speeds (in km/s) corresponding to the spacecraft/bodies, but you can. If `vsw_list` is not provided (or as an empty list, `vsw_list=[]`), it is automatically tried to obtain measured solar wind speeds per spacecraft. If this does not succeed for a spacecraft, a default solar wind speed of 400 km/s is assumed (this can be adjusted with `default_vsw`, e.g., `default_vsw=300`).

In [ ]:
body_list = ['Earth', 'Solar Orbiter', 'PSP']
date = '2022-3-1 12:00:00'

Initialize the SolarMACH object for these options:

In [ ]:
sm1 = SolarMACH(date=date, body_list=body_list)

No solar wind speeds defined, trying to obtain measurements...
Using 'ACE' measurements for 'Earth'.


And produce the final plot:

In [ ]:
sm1.plot(plot_sun_body_line=True)

This plot shows a view from the top on the ecliptic plane with the Sun in the center and the Earth (indicated by green symbol) at "6 o'clock". The solid lines give estimations of single field lines of an ideal Parker heliospheric magnetic field connecting the corresponding observers to the Sun, while the dashed lines just indicate the line of sight from each of them to the Sun.

If you do not want or need that the solar wind speeds are obtained from online sources, you can manually define them with `vsw_list`. 
Note that each speed corresponds position-wise with the corresponding spacecraft/bodies list:

In [ ]:
vsw_list = [400, 400, 400]    # position-sensitive solar wind speed per body in body_list
body_list = ['Earth', 'Solar Orbiter', 'PSP']
date = '2022-3-1 12:00:00'

sm1a = SolarMACH(date=date, body_list=body_list, vsw_list=vsw_list)
sm1a.plot(plot_sun_body_line=True)

---

## 2. Example with all the details <a class="anchor" id="Example_all_details"></a>

First, get a list of available bodies/spacecraft:

In [ ]:
print(print_body_list().index)

Index(['Earth', 'ACE', 'BepiColombo', 'Cassini', 'Europa Clipper', 'JUICE',
       'Juno', 'Jupiter', 'L1', 'Mars', 'Mars Express', 'MAVEN', 'Mercury',
       'MESSENGER', 'PSP', 'Pioneer10', 'Pioneer11', 'Rosetta', 'SOHO',
       'Solar Orbiter', 'STEREO B', 'STEREO A', 'Ulysses', 'Venus', 'Voyager1',
       'Voyager2', 'WIND'],
      dtype='object', name='Key')


Provide the necessary options, this time for more spacecraft:

In [ ]:
body_list = ['Mercury', 'Venus', 'Earth', 'Mars', 'STEREO A', 'STEREO B', 'Solar Orbiter', 'PSP', 'BepiColombo']
date = '2021-6-1 12:00:00'

Leave the position-sensitive solar wind speed list empty, so that they are obtained from actual measurements, and define the solar wind speed that should be used if no measurements can be obtained for a spacecraft:

In [ ]:
vsw_list = []
default_vsw = 350  # km/s

The default coordinate system is [Carrington coordinates](https://docs.sunpy.org/en/stable/generated/api/sunpy.coordinates.frames.HeliographicCarrington.html), alternatively one could select the Earth-centered [Stonyhurst coordinate](https://docs.sunpy.org/en/stable/generated/api/sunpy.coordinates.frames.HeliographicStonyhurst.html) system:

In [ ]:
coord_sys = 'Carrington'     # 'Carrington' (default) or 'Stoneyhurst'

Now we also want to indicate the position and direction of a flare, and the (assumed) solar wind speed at its location:

In [ ]:
reference_long = 0            # Carrington longitude of reference (None to omit)
reference_lat = 0             # Carrington latitude of reference (None to omit)
reference_vsw = 400           # define solar wind speed at reference in km/s

In addition, we explicitly provide all availabe plotting options:

In [ ]:
plot_spirals = True           # plot Parker spirals for each body
plot_sun_body_line = False    # plot straight line between Sun and body
long_offset = 0               # longitudinal offset for polar plot; defines where Earth's longitude is (by default 270, i.e., at "6 o'clock")
transparent = False           # make output figure background transparent
markers = 'numbers'           # use 'numbers' or 'letters' for the body markers (use False for colored squares)
filename = f'Solar-MACH_{date.replace(" ", "_")}.png'  # define filename of output figure

Finally, initializing and plotting with these options. If `outfile` is provided, the plot will be saved next to the Notebook with the provided `filename`.

In [ ]:
sm2 = SolarMACH(date=date, body_list=body_list, vsw_list=vsw_list,
                reference_long=reference_long, reference_lat=reference_lat,
                coord_sys=coord_sys)

sm2.plot(plot_spirals=plot_spirals,
         plot_sun_body_line=plot_sun_body_line,
         long_offset=long_offset,
         reference_vsw=reference_vsw,
         transparent=transparent,
         markers=markers,
         outfile=filename
         )

/Users/jagies/miniforge3/envs/serpentine/lib/python3.12/site-packages/speasy/core/data_containers.py:17: UserWarning: no explicit representation of timezones available for np.datetime64
  return np.searchsorted(time, np.datetime64(key, 'ns'), side='left')
2025-05-06 11:52:35 - speasy.core.dataprovider - WARNING: You are requesting vpbulk_stb outside of its definition range <DateTimeRange: 2007-03-01T00:00:12+00:00 -> 2014-10-31T23:59:30+00:00>


No solar wind speeds defined, trying to obtain measurements...
Body 'Mercury' not supported, assuming default Vsw value of 400.0 km/s.
Body 'Venus' not supported, assuming default Vsw value of 400.0 km/s.
Using 'ACE' measurements for 'Earth'.
Body 'Mars' not supported, assuming default Vsw value of 400.0 km/s.
No Vsw data found for 'STEREO B' on 2021-06-01 12:00:00, assuming default Vsw value of 400.0 km/s.
No Vsw data found for 'Solar Orbiter' on 2021-06-01 12:00:00, assuming default Vsw value of 400.0 km/s.
Body 'BepiColombo' not supported, assuming default Vsw value of 400.0 km/s.


All the data can also be obtained as a Pandas DataFrame for further use. Note that here you can also see the actually measured solar wind speeds at some spacecraft (`Vsw`):

In [ ]:
df = sm2.coord_table
display(df)

In [ ]:
df['Heliocentric distance (AU)'].values

array([0.45657717, 0.71866253, 1.0140839 , 1.65743876, 0.9633026 ,
       1.08601488, 0.95207518, 0.7155775 , 0.80726552])

---

## 3. Example using Stonyhurst coordinates for reference <a class="anchor" id="Example_stonyhurst"></a>

Let's take a look at the situation at the first ground-level enhancement (GLE) of solar cycle 25 on 28 October 2021 

First, we just provide some options as before:

In [ ]:
body_list = ['STEREO-A', 'Earth', 'BepiColombo', 'PSP', 'Solar Orbiter', 'Mars']
vsw_list = [340, 300, 350, 350, 320, 350]        # position-sensitive solar wind speed per body in body_list
date = '2021-10-28 15:20:00'
coord_sys = 'Stoneyhurst'

# optional parameters
plot_spirals = True           # plot Parker spirals for each body
plot_sun_body_line = True     # plot straight line between Sun and body
transparent = False           # make output figure background transparent
markers = 'letters'           # use 'numbers' or 'letters' for the body markers (use False for colored squares)
filename = f'Solar-MACH_{date.replace(" ", "_")}.png'  # define filename of output figure

But now we want to provide the coordinates of the flare in [Stonyhurst coordinates](https://docs.sunpy.org/en/stable/generated/api/sunpy.coordinates.frames.HeliographicStonyhurst.html) (instead of [Carrington](https://docs.sunpy.org/en/stable/generated/api/sunpy.coordinates.frames.HeliographicCarrington.html)).

In [ ]:
reference_long = 2            # Stonyhurst longitude of reference (None to omit)
reference_lat = 26            # Stonyhurst latitude of reference (None to omit)
reference_vsw = 300           # define solar wind speed at reference

Finally, initializing and plotting with these options:

In [ ]:
sm3 = SolarMACH(date=date, body_list=body_list, vsw_list=vsw_list,
                reference_long=reference_long, reference_lat=reference_lat,
                coord_sys=coord_sys)
sm3.plot(plot_spirals=plot_spirals,
         plot_sun_body_line=plot_sun_body_line,
         reference_vsw=reference_vsw,
         transparent=transparent,
         markers=markers,
         outfile=filename
         )

---
## 4. Only obtain data as Pandas DataFrame <a class="anchor" id="only_dataframe"></a>

We can also just obtain a table with the spatial data, without producing a plot at all.

First provide necessary options:

In [ ]:
body_list = ['STEREO-A', 'Earth', 'BepiColombo', 'PSP', 'Solar Orbiter', 'Mars']
vsw_list = []  # leave empty to obtain measurements
date = '2022-6-1 12:00:00'

Then initialize `SolarMACH` and obtain data as Pandas DataFrame:

In [ ]:
sm4 = SolarMACH(date=date, body_list=body_list, vsw_list=vsw_list)
df = sm4.coord_table
display(df)

No solar wind speeds defined, trying to obtain measurements...
Using 'ACE' measurements for 'Earth'.
Body 'BepiColombo' not supported, assuming default Vsw value of 400.0 km/s.


/Users/jagies/miniforge3/envs/serpentine/lib/python3.12/site-packages/speasy/core/data_containers.py:17: UserWarning: no explicit representation of timezones available for np.datetime64
  return np.searchsorted(time, np.datetime64(key, 'ns'), side='left')


No Vsw data found for 'Parker Solar Probe' on 2022-06-01 12:00:00, assuming default Vsw value of 400.0 km/s.
No Vsw data found for 'Solar Orbiter' on 2022-06-01 12:00:00, assuming default Vsw value of 400.0 km/s.
Body 'Mars' not supported, assuming default Vsw value of 400.0 km/s.


If we also provide the `reference` information, it will be available in the table, too:

In [ ]:
sm4 = SolarMACH(date=date, body_list=body_list, vsw_list=vsw_list,
                reference_long=273, reference_lat=7)
df = sm4.coord_table
display(df)

No solar wind speeds defined, trying to obtain measurements...
Using 'ACE' measurements for 'Earth'.
Body 'BepiColombo' not supported, assuming default Vsw value of 400.0 km/s.
No Vsw data found for 'Parker Solar Probe' on 2022-06-01 12:00:00, assuming default Vsw value of 400.0 km/s.
No Vsw data found for 'Solar Orbiter' on 2022-06-01 12:00:00, assuming default Vsw value of 400.0 km/s.
Body 'Mars' not supported, assuming default Vsw value of 400.0 km/s.


/Users/jagies/miniforge3/envs/serpentine/lib/python3.12/site-packages/speasy/core/data_containers.py:17: UserWarning: no explicit representation of timezones available for np.datetime64
  return np.searchsorted(time, np.datetime64(key, 'ns'), side='left')


Note again that by default all coordinates are given in Carrington coordinates (as indicated in the column titles). To have this data in Stoneyhurst coordinates, define the `coord_sys` when initilizing `SolarMACH`:

In [ ]:
sm4a = SolarMACH(date=date, body_list=body_list, vsw_list=vsw_list, coord_sys='Stoneyhurst')
df = sm4a.coord_table
display(df)

No solar wind speeds defined, trying to obtain measurements...
Using 'ACE' measurements for 'Earth'.
Body 'BepiColombo' not supported, assuming default Vsw value of 400.0 km/s.
No Vsw data found for 'Parker Solar Probe' on 2022-06-01 12:00:00, assuming default Vsw value of 400.0 km/s.
No Vsw data found for 'Solar Orbiter' on 2022-06-01 12:00:00, assuming default Vsw value of 400.0 km/s.
Body 'Mars' not supported, assuming default Vsw value of 400.0 km/s.


/Users/jagies/miniforge3/envs/serpentine/lib/python3.12/site-packages/speasy/core/data_containers.py:17: UserWarning: no explicit representation of timezones available for np.datetime64
  return np.searchsorted(time, np.datetime64(key, 'ns'), side='left')


---
## 5. Advanced plotting options <a class="anchor" id="plotting_options"></a>

Since version 0.1.6, `solarmach` provides the option to color a cone-shaped area or an area between two Parker spirals in the heliographic plane. For this, `SolarMACH.plot()` has three new options:

- **long_sector**: list of 2 numbers: start and stop longitude of a shaded area; e.g. `[350, 20]` to get a cone from 350 to 20 degree longitude (for long_sector_vsw=None`).
- **long_sector_vsw**: list of 2 numbers: Solar wind speed used to calculate Parker spirals (at start and stop longitude provided by `long_sector`) between which a reference cone should be drawn; e.g. `[400, 400]` to assume for both edges of the fill area a Parker spiral produced by solar wind speeds of 400 km/s. If `None`, instead of Parker spirals, straight lines are used, i.e. a simple cone will be plotted. By default, `None`.
- **long_sector_color**: String defining the matplotlib color used for the shading defined by `long_sector`. By default, `'red'`.

In addition, it is now possible to plot a set of evenly distributed Parker spirals in the background, see [5.3 Example 3: plot background Parker spirals](#5.3-Example-3:-plot-background-Parker-spirals).

### 5.1 Example 1: plot cone-shaped area

In [ ]:
body_list = ['STEREO-A', 'Earth', 'BepiColombo', 'PSP', 'Solar Orbiter']
vsw_list = [400, 400, 400, 400, 400]
date = '2022-3-11 12:00:00'

sm5a = SolarMACH(date=date, body_list=body_list, vsw_list=vsw_list)
sm5a.plot(long_sector=[290, 328], long_sector_vsw=None, long_sector_color='red')

### 5.2 Example 2: shade area between two Parker spirals

In [ ]:
body_list = ['STEREO-A', 'Earth', 'BepiColombo', 'PSP', 'Solar Orbiter']
vsw_list = [400, 400, 400, 400, 400]
date = '2022-3-11 12:00:00'

sm5b = SolarMACH(date=date, body_list=body_list, vsw_list=vsw_list)
sm5b.plot(long_sector=[290, 328], long_sector_vsw=[400,600], long_sector_color='cyan')

Note that there still is a bug if the speeds in `long_sector_vsw` differ to some extent; then the plotting might not work as intended.

### 5.3 Example 3: plot background Parker spirals

Since version 0.1.6, `solarmach` provides the option to plot a set of evenly distributed Parker spirals in the background, using the option `background_spirals` in `SolarMACH.plot()`:

**background_spirals**: list of 2 numbers (and 3 optional strings). If defined, plot evenly distributed Parker spirals over 360°. `background_spirals[0]` defines the number of spirals, `background_spirals[1]` the solar wind speed in km/s used for their calculation. 

Optionally, `background_spirals[2]`, `background_spirals[3]`, and `background_spirals[4]` change the plotting line style, color, and alpha setting, respectively (default values `':'`, `'grey'`, and `0.1`). 

In [ ]:
body_list = ['STEREO-A', 'Earth', 'BepiColombo', 'PSP', 'Solar Orbiter']
vsw_list = [400, 400, 400, 400, 400]
date = '2022-3-11 12:00:00'

sm = SolarMACH(date=date, body_list=body_list, vsw_list=vsw_list)
sm.plot(background_spirals=[6, 600])

In [ ]:
body_list = ['STEREO-A', 'Earth', 'BepiColombo', 'PSP', 'Solar Orbiter']
vsw_list = [400, 400, 400, 400, 400]
date = '2022-3-11 12:00:00'

sm = SolarMACH(date=date, body_list=body_list, vsw_list=vsw_list)
sm.plot(background_spirals=[6, 600, '-', 'red', 0.5])

---
## 6. Advanced: edit the figure <a class="anchor" id="edit_figure"></a>
Let's produce again the figure for the Oct 2021 GLE event.

In [ ]:
body_list = ['STEREO-A', 'Earth', 'BepiColombo', 'PSP', 'Solar Orbiter', 'Mars']
vsw_list = [340, 300, 350, 350, 320, 350]  # position-sensitive solar wind speed per body in body_list
date = '2021-10-28 15:20:00'
reference_long = 130           # Carrington longitude of reference (None to omit)
reference_lat = 0             # Carrington latitude of reference (None to omit)
reference_vsw = 400           # define solar wind speed at reference
sm6 = SolarMACH(date=date, body_list=body_list, vsw_list=vsw_list,
                reference_long=reference_long, reference_lat=reference_lat)

But now we prodive the option `return_plot_object=True` to the `plot()` call, so that the matplotlib figure and axis object will be returned:

In [ ]:
fig, ax = sm6.plot(plot_spirals=True,
                   plot_sun_body_line=True,
                   transparent=False,
                   markers='numbers',
                   return_plot_object=True
                  )

Now we can make some post-adjustments to the figure (requires of course some Python knowledge):

In [ ]:
ax.set_title("What's that?! A new title!", pad=60)

Text(0.5, 1.0, "What's that?! A new title!")

In [ ]:
fig

We can also add another red reference arrow to the figure, e.g., to indicate another flare:

In [ ]:
import numpy as np

new_reference_long = 60

arrow_dist = min([sm6.max_dist/3.2, 2.])  # arrow length
ax.arrow(np.deg2rad(new_reference_long), 0.01, 0, arrow_dist, head_width=0.2, head_length=0.07, edgecolor='red', facecolor='red', lw=1.8, zorder=5, overhang=0.2)
ax.set_rmax(sm6.max_dist + 0.3)  # re-set max. radial dist. of plot

In [ ]:
fig

---
## 7. Ideas for further usage <a class="anchor" id="ideas_further_usage"></a>

### 7.1 Loop over multiple datetimes (plots) <a class="anchor" id="loop_datetimes_plots"></a>

This might be useful to either:

- read-in an *event catalog*, and loop over those datetimes to quickly get the constellations of all these events,
- create a series of daily constellation plots, and combine them into one animation:

In [ ]:
body_list = ['Mercury', 'Venus', 'Earth', 'Mars', 'STEREO A', 'STEREO B', 'Solar Orbiter', 'PSP', 'BepiColombo']
vsw_list = len(body_list) * [350]   # position-sensitive solar wind speed per body in body_list
plot_spirals = True                 # plot Parker spirals for each body
plot_sun_body_line = False          # plot straight line between Sun and body
transparent = False                 # make output figure background transparent
markers = 'numbers'                 # use 'numbers' or 'letters' for the body markers (use False for colored squares)

for i in range(2,19,1):    
    j = str(i).rjust(2, '0')
    date = f'2022-6-{j} 12:00:00'
    filename = f'animate_{date[:-9]}.png'        # define filename of output figure

    sm7 = SolarMACH(date=date, body_list=body_list, vsw_list=vsw_list, coord_sys='Stoneyhurst')
    sm7.plot(plot_spirals=plot_spirals,
             plot_sun_body_line=plot_sun_body_line,
             transparent=transparent,
             markers=markers,
             outfile=filename
             )

Get a sorted list of the files just created using `glob`:

In [ ]:
import glob
files = sorted(glob.glob(filename.replace(f'{i}', '*')))

Build an animated GIF out of these files using `imageio`:

In [ ]:
import imageio
images_data = []
for filename in files:
    data = imageio.v2.imread(filename)
    images_data.append(data)

# write to animated gif; duration (in ms) defines how fast the animation is.
imageio.mimwrite('solarmach.gif', images_data, format= '.gif', duration=100, loop=0)

When completed, this will have created an animated GIF file `solarmach.gif` in the same directory as this notebook (as well as a series of files `animate_DATE.png` that can be deleted).

### 7.2 Loop over multiple datetimes (only data) <a class="anchor" id="loop_datetimes_data"></a>

For example, to look for spacecraft alignments, like *"When are PSP and Solar Orbiter at the same magnetic footpoint?"*

In [ ]:
body_list = ['Mercury', 'Venus', 'Earth', 'Mars', 'STEREO A', 'Solar Orbiter', 'PSP', 'BepiColombo']
vsw_list = len(body_list) * [350]  # position-sensitive solar wind speed per body in body_list

df = []
dates = []
for i in range(1,31,1):
    date = f'2022-6-{i} 12:00:00'

    sm7b = SolarMACH(date=date, body_list=body_list, vsw_list=vsw_list)
    df = df + [sm7b.coord_table]
    dates = dates + [date]

In [ ]:
display(df[0])
display(df[1])
display(df[2])

In [ ]:
display(dates)

['2022-6-1 12:00:00',
 '2022-6-2 12:00:00',
 '2022-6-3 12:00:00',
 '2022-6-4 12:00:00',
 '2022-6-5 12:00:00',
 '2022-6-6 12:00:00',
 '2022-6-7 12:00:00',
 '2022-6-8 12:00:00',
 '2022-6-9 12:00:00',
 '2022-6-10 12:00:00',
 '2022-6-11 12:00:00',
 '2022-6-12 12:00:00',
 '2022-6-13 12:00:00',
 '2022-6-14 12:00:00',
 '2022-6-15 12:00:00',
 '2022-6-16 12:00:00',
 '2022-6-17 12:00:00',
 '2022-6-18 12:00:00',
 '2022-6-19 12:00:00',
 '2022-6-20 12:00:00',
 '2022-6-21 12:00:00',
 '2022-6-22 12:00:00',
 '2022-6-23 12:00:00',
 '2022-6-24 12:00:00',
 '2022-6-25 12:00:00',
 '2022-6-26 12:00:00',
 '2022-6-27 12:00:00',
 '2022-6-28 12:00:00',
 '2022-6-29 12:00:00',
 '2022-6-30 12:00:00']

---

## 8. Interactive 3D figure <a class="anchor" id="plot_3d"></a>

Solar-MACH can also provide a fully interactive 3D view of the heliospheric observer situation. This is especially interesting for out of the ecliptic missions like Ulysses or Solar Orbiter.

The initialization is the same as for the classic Solar-MACH plot:

In [ ]:
date = '2030-7-1 13:00:00'
body_list = ['Earth', 'Mars', 'Venus', 'Solar Orbiter']
vsw_list = len(body_list) * [350]
sm8 = SolarMACH(date=date, body_list=body_list, vsw_list=vsw_list)

/Users/jagies/miniforge3/envs/serpentine/lib/python3.12/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
/Users/jagies/miniforge3/envs/serpentine/lib/python3.12/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
/Users/jagies/miniforge3/envs/serpentine/lib/python3.12/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
/Users/jagies/miniforge3/envs/serpentine/lib/python3.12/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)


Now, instead of `sm8.plot()`, `sm8.plot_3d()` is used. It has partly the same options, but also some new ones:


In [ ]:
sm8.plot_3d?

The resulting figure can be tilted/rotated (panned) by pressing the left (right) mouse botton and moving it. Mouse wheel zooms in and out.

In [ ]:
fig = sm8.plot_3d(plot_spirals=True, plot_sun_body_line=True, plot_vertical_line=True, markers=False,
                  plot_equatorial_plane=True, plot_3d_grid=False, reference_vsw=400, return_plot_object=True)

/Users/jagies/miniforge3/envs/serpentine/lib/python3.12/site-packages/erfa/core.py:133: ErfaWarning:

ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"



Unfortunately, the integrated function to save the figure as PNG only saves the starting view. If you adjusted the view with the mouse, you have to make a screenshot to save it as of now.

But the interactive 3D figure can be saved and shared as an HTML file. Saving it with `include_plotlyjs='cdn'` requires an internet connection when openining, but the file size will be smaller. `include_plotlyjs=True` increases the file size, but returns an offline-viewable HTML file.


In [ ]:
fig.write_html("solarmach.html", include_plotlyjs='cdn')

---

## 9. Further backmapping with PFSS <a class="anchor" id="pfss"></a>

A recent extension to Solar-MACH is the ability to further extend the magnetic connection of an observer to the close vicinity of the Sun using a [Potential Field Source Surface (PFSS)](https://nso.edu/data/nisp-data/pfss/) model that connects the heliospheric magnetic field (HMF) to the solar corona. 

Let's first load again Solar-MACH for the April 17, 2021 event. We will use the outputs of it as inputs for the PFSS model.

Note that we do not pre-define the solar wind speeds per spacecraft with `vsw_list`, but use the interal function to try to obtain the actual measured values.

In [ ]:
date = '2021-4-17 13:00:00'
body_list = ['STEREO-A', 'PSP', 'Earth', 'BepiColombo', 'Solar Orbiter']
coord_sys = 'Carrington'  # 'Stonyhurst' or 'Carrington'

sm9 = SolarMACH(date=date, body_list=body_list, coord_sys=coord_sys)

No solar wind speeds defined, trying to obtain measurements...
Using 'ACE' measurements for 'Earth'.
Body 'BepiColombo' not supported, assuming default Vsw value of 400.0 km/s.


/Users/jagies/miniforge3/envs/serpentine/lib/python3.12/site-packages/speasy/core/data_containers.py:17: UserWarning: no explicit representation of timezones available for np.datetime64
  return np.searchsorted(time, np.datetime64(key, 'ns'), side='left')


### First acquire a GONG (Global Oscillation Network Group) map of the Sun's surface

In [ ]:
gong_map = get_gong_map(time=date, filepath=None)

Automatic file search based on given time found /Users/jagies/uni/serpentine/solarmach/examples/mrzqs210417t1304c2243_256.fits.gz


In [ ]:
display(gong_map)

2025-05-06 12:18:00 - sunpy - INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere.


INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]


### Based on the map and a boundary condition, calculate a pfss solution

Note that depending on the computer used, this process can take up to a few minutes.

In [ ]:
# Set the height of the source surface as a boundary condition for pfss extrapolation
rss = 2.5

Note that you *have to* define the coordinate system to be used. This has to be the same one as for initializing `SolarMACH(date=date, body_list=body_list, coord_sys=coord_sys)`; the default for that is `coord_sys='Carrington'`. The system will complain while plotting if the coordinate systems do not agree.

In [ ]:
# Calculate the potential field source surface solution. 
pfss_solution = calculate_pfss_solution(gong_map=gong_map, rss=rss, coord_sys=coord_sys)

2025-05-06 12:18:03 - reproject.common - INFO: Calling _reproject_full in non-dask mode
2025-05-06 12:18:03 - sunpy - INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere.


INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]


### Generate a semi-logarithmic PFSS plot (can take a minute)
Note that the output figure combines linear and logarithmic scaling for the radial axis: Up to the PFSS at (usually) 2.5 solar radii the figure uses a linear scaling, whereas the main part of the heliosphere is shown with a logarithmic scale in radial distance. This allows to highlight the magnetic connections close to the Sun, while maintaining the connection further out in interplanetary space. The color coding below the PFSS indicates the corresponding latitudes.

>Note: 'vary' is a boolean variable that controls if a flux rope should also be calculated around the point at which ballistic backmapping intersects with the source surface. Enabling this option increases computation time.

In [ ]:
sm9.plot_pfss(rss=rss, pfss_solution=pfss_solution, markers='numbers', vary=True)

  0%|          | 0/5 [00:00<?, ?it/s]

And obtain the data as a Pandas DataFrame:

In [ ]:
display(sm9.coord_table)

### Look into the sub-pfss field lines with 3D view

In [ ]:
fig = sm9.pfss_3d(color_code="object", return_plot_object=True)

Save interactive 3D figure as an HTML file that can be shared. Saving it with `include_plotlyjs='cdn'` requires an internet connection when openining, but the file size will be smaller. `include_plotlyjs=True` increases the file size, but returns an offline-viewable HTML file.

In [ ]:
fig.write_html("solarmach.html", include_plotlyjs='cdn')